In [6]:
import EegFunctions
import importlib
importlib.reload(EegFunctions)
from EegFunctions import *
%matplotlib qt

In [7]:
class Config:
    DATA_FILE =  r'C:\Users\Keerti\Downloads\MySolution\PlayBooks\data\10_MMN_preprocessed.set'
    DEVICE = 'unicorn'  # 'muse' or 'unicorn'
    # SAMPLING_RATE = 250
    CHANNELS = ['TP9', 'AF7', 'AF8', 'TP10'] if DEVICE == 'muse' else ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8']
    ANALYSIS_CHANNELS = ['TP9', 'TP10'] if DEVICE == 'muse' else [ 'Pz','Cz']
    EPOCH_TMIN = -0.2
    EPOCH_TMAX = 0.8
    BASELINE = (-0.2, 0)
    HIGH_PASS = None
    LOW_PASS = 30.0  

In [8]:
raw,df = create_mne_raw(Config.DATA_FILE)
montage = mne.channels.make_standard_montage('standard_1020')
eog_channels = ['HEOG-left', 'HEOG-right', 'VEOG-lower', 'HEOG-bipolar', 'VEOG-bipolar']
channel_types = {ch: 'eog' for ch in eog_channels}
raw.set_channel_types(channel_types)
raw.set_montage(montage, match_case=False)

raw_filtered = raw.copy()
raw_filtered.filter(
        l_freq=Config.HIGH_PASS, 
        h_freq=Config.LOW_PASS, 
        h_trans_bandwidth=6,
        method='fir',
    )
events,event_ids,stimulus_count = extract_events(raw=raw_filtered)
Config.EVENT_DICT = event_ids
epochs = create_epochs(raw_filtered, events,Config)
bin_1_idx, bin_2_idx = create_bins(events)
print(f"Bin 1 (Deviant preceded by Standard) trials: {len(bin_1_idx)}")
print(f"Bin 2 (Standard preceded by Standard) trials: {len(bin_2_idx)}")
epochs_bin1 = epochs[bin_1_idx]  # Deviant preceded by standard
epochs_bin2 = epochs[bin_2_idx]
# all_epochs_contat = mne.concatenate_epochs([epochs_bin2, epochs_bin1])
print(len(epochs_bin1), len(epochs_bin2))

=== CREATING MNE OBJECT ===
Number of trials detected: 1
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 6.00 Hz (-6 dB cutoff frequency: 33.00 Hz)
- Filter length: 141 samples (0.551 s)

=== EXTRACTING EVENTS ===

Found 1001 annotations:
  1: 1 occurrences
  180: 15 occurrences
  70: 200 occurrences
  80: 785 occurrences
Used Annotations descriptions: [np.str_('1'), np.str_('180'), np.str_('70'), np.str_('80')]
=== CREATING EPOCHS ===
Not setting metadata
1001 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1001 events and 257 original time points ...
0 bad epochs dropped
1: 1
180: 15
70: 200

In [ ]:
epochs.plot(events=True,event_id= True,n_epochs=4)

In [9]:
bad_epochs = moving_window_step_artifact_detection(epochs,
                                                    channel_name="VEOG-bipolar", 
                                                    window_size=0.2, step_size=0.01, threshold=50)
bad_epochs_movement = moving_window_step_artifact_detection(epochs,
                                                    channel_name="HEOG-bipolar", 
                                                    window_size=0.2, step_size=0.01, threshold=32)
epochs_blinks = epochs.copy()
epochs_movement = epochs.copy()
epochs_blinks.drop_log = [None] * len(epochs)
epochs_movement.drop_log = [None] * len(epochs)

for i, is_bad in enumerate(bad_epochs):
    if is_bad:
        epochs_blinks.drop_log[i] = "BLINK_ARTIFACT"

for i, is_bad in enumerate(bad_epochs_movement):
    if is_bad:
        epochs_movement.drop_log[i] = "EYE_MOVEMENT_ARTIFACT"

rejection_rate = np.where(bad_epochs)[0].shape[0] / len(epochs) * 100
print(f"Trials flagged for rejection: {np.where(bad_epochs)[0].shape[0]} ({rejection_rate:.1f}%)")

rejection_rate = np.where(bad_epochs_movement)[0].shape[0] / len(epochs) * 100
print(f"Trials flagged for rejection: {np.where(bad_epochs_movement)[0].shape[0]} ({rejection_rate:.1f}%)")

# plot_bad_epochs(epochs,bad_epochs,'HEOG-bipolar',n_epochs_to_plot=15,title="Blink Artifact Detection")

Trials flagged for rejection: 314 (31.4%)
Trials flagged for rejection: 354 (35.4%)


In [188]:
epochs_blinks = epochs_blinks.drop(np.where(bad_epochs)[0])
epochs_movement = epochs_movement.drop(np.where(bad_epochs_movement)[0])
# epochs_movement.drop_bad();
epoch_lists = [epochs_blinks,epochs_movement]
time_window=(-0.125, 0.225)
sme_results = compute_sme_erplab(epoch_lists,time_window)

Dropped 314 epochs: 1, 6, 7, 9, 10, 16, 17, 20, 21, 23, 24, 25, 26, 27, 28, 33, 34, 36, 37, 40, 41, 44, 45, 48, 49, 56, 57, 60, 61, 62, 66, 67, 71, 72, 75, 76, 81, 82, 83, 84, 89, 90, 92, 93, 97, 98, 102, 103, 120, 121, 124, 125, 132, 133, 138, 139, 146, 147, 149, 150, 157, 159, 160, 163, 164, 167, 168, 171, 172, 173, 174, 190, 191, 197, 198, 200, 201, 207, 208, 215, 216, 218, 219, 220, 221, 230, 231, 235, 236, 240, 241, 252, 253, 258, 259, 261, 262, 265, 266, 275, 276, 279, 280, 285, 286, 295, 296, 303, 304, 308, 309, 316, 317, 321, 322, 330, 331, 336, 337, 344, 345, 347, 348, 358, 359, 360, 361, 365, 366, 373, 374, 379, 380, 385, 386, 387, 390, 391, 394, 395, 399, 400, 404, 405, 412, 413, 417, 418, 419, 420, 425, 426, 435, 436, 444, 445, 452, 453, 460, 461, 465, 466, 473, 474, 479, 480, 481, 482, 491, 492, 496, 497, 506, 507, 514, 515, 527, 528, 535, 536, 542, 543, 547, 548, 556, 557, 562, 563, 566, 572, 573, 576, 577, 590, 591, 598, 599, 605, 606, 609, 610, 615, 616, 617, 626, 627, 

In [189]:
import numpy as np

for ep in [epochs_blinks, epochs_movement]:
    epochs_bc = ep.copy().apply_baseline((-0.2, 0))
    ch_name = 'FCz'
    tmin, tmax = 0.125, 0.225  # 125–225 ms window
    mask = (epochs_bc.times >= tmin) & (epochs_bc.times <= tmax)

    bins = list(epochs_bc.event_id.keys())[2::]  
    aSME_dict = {}
    for b in bins:
        data = epochs_bc[b].get_data()[:, epochs_bc.ch_names.index(ch_name), :] * 1e6  # volts → μV
        trial_means = data[:, mask].mean(axis=1)  # mean per trial in window
        aSME = trial_means.std() / np.sqrt(trial_means.size)  # standard error across trials
        aSME_dict[b] = aSME

    for b, val in aSME_dict.items():
        print(f"aSME FCz 125–225 ms ({b}): {val:.4f} μV")

Applying baseline correction (mode: mean)
aSME FCz 125–225 ms (70): 0.8276 μV
aSME FCz 125–225 ms (80): 0.4881 μV
Applying baseline correction (mode: mean)
aSME FCz 125–225 ms (70): 0.9554 μV
aSME FCz 125–225 ms (80): 0.5163 μV


In [137]:
import matplotlib.pyplot as plt
from mne.viz import plot_compare_evokeds

# Average epochs per condition
eog_channels = ["VEOG-bipolar", "VEOG-lower","HEOG-bipolar"]
bad_indices = [i for i, reason in enumerate(epochs.drop_log) if reason]
epochs_plot = epochs.copy()
# epochs_plot = epochs[bad_indices].copy()
# epochs_plot = epochs.copy();epochs_plot.drop_bad()
epochs_plot.set_channel_types({ch: "eeg" for ch in eog_channels})

evokeds = epochs_plot.average(by_event_type=True,picks='all')

# Channels to plot individually
channels = ["FP2", "VEOG-lower", "VEOG-bipolar", "FCz","HEOG-bipolar","FP1"]
n_channels = len(channels)

# Create subplots
fig, axes = plt.subplots(3, 2, figsize=(4, 2.5 * n_channels), sharex=False)

# Plot each channel separately using plot_compare_evokeds
for ax, ch_name in zip(axes.flatten(), channels):
    cnt = 0
    plot_compare_evokeds(
        {"standard": evokeds[3], "deviant": evokeds[2]},
        picks=[ch_name],
        axes=[ax],
        combine=None,  # plot each channel individually
        colors={"standard": "blue", "deviant": "red"},
        truncate_yaxis=False,
        truncate_xaxis=False,
        # ylim={'eeg':[-10,10]}
    )
    ax.set_title(ch_name)
    cnt += 1
    # ax.set_ylim(-20, 20)

# axes[-1].set_xlabel("Time (s)")
plt.tight_layout()
plt.show()


C:\Users\Keerti\AppData\Local\Temp\ipykernel_16020\434201061.py:24: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  plot_compare_evokeds(
C:\Users\Keerti\AppData\Local\Temp\ipykernel_16020\434201061.py:24: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  plot_compare_evokeds(
C:\Users\Keerti\AppData\Local\Temp\ipykernel_16020\434201061.py:24: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  plot_compare_evokeds(
C:\Users\Keerti\AppData\Local\Temp\ipykernel_16020\434201061.py:39: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
